In [1]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/cameronfaulkner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cameronfaulkner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dataset = pd.read_csv("/Users/cameronfaulkner/Documents/Cogs 118a/Final Project/Sentiment Added Dataset")
dataset.head()

,rawContent,replyCount,retweetCount,likeCount,quoteCount,links,media,mentionedUsers,hashtags,Party,Positive,Negative,Neutral
0,New Year’s Day in Seward overlooking the harbo...,44.0,12.0,239.0,2.0,0,3,0,0,1,1,0,0
1,Happy #NewYear!,19.0,4.0,22.0,1.0,0,0,0,1,1,1,0,0
2,I championed provisions to strengthen Alaska’s...,9.0,5.0,19.0,0.0,0,0,0,0,1,1,0,0
3,"December: As senior Appropriator, I helped wri...",11.0,6.0,28.0,3.0,0,1,0,0,1,1,0,0
4,December: Congressman Don Young’s legacy will ...,5.0,9.0,46.0,1.0,0,1,0,0,1,1,0,0


Processing Test to Maximize Classifier Performance.

In [60]:
def text_preprocess(ds: pd.Series) -> pd.Series:
    for m in range(len(ds)):
        
        main_words = re.sub('[^a-zA-Z]', ' ', ds[m])                                      # Retain only alphabets
        main_words = (main_words.lower()).split()
        main_words = [w for w in main_words if not w in set(stopwords.words('english'))]  # Remove stopwords
        
        lem = WordNetLemmatizer()
        main_words = [lem.lemmatize(w) for w in main_words if len(w) > 1]                 # Group different forms of the same word
        
        main_words = ' '.join(main_words)
        ds[m] = main_words
    return ds

test_processed = text_preprocess(dataset['rawContent'].values)

In [63]:
test_processed.shape

(786268,)

In [64]:
dataset["processed text"] = test_processed

In [66]:
#save dataframe with processed data
dataset.to_csv("Processed Data",index=False)

In [3]:
dataset["processed text"]

0         new year day seward overlooking harbor beautif...
1                                             happy newyear
2         championed provision strengthen alaska economy...
3         december senior appropriator helped write majo...
4         december congressman young legacy live many wa...
                                ...                        
786263    violent mob assault capitol attempt prevent ca...
786264    proud stand richard hudson protect second amen...
786265    democrat repeatedly abused power achieve polit...
786266    congratulate new colleague sworn office today ...
786267    honored sworn today represent great people wyo...
Name: processed text, Length: 786268, dtype: object

Vectorize Text

In [6]:
X = dataset["processed text"]
td = TfidfVectorizer(max_features = 4500)
X = td.fit_transform(X.apply(lambda x: np.str_(x))).toarray()# v.fit_transform(df['Review'].apply(lambda x: np.str_(x)))
print(X.shape)

(786268, 4500)


Add Relevant Non-Text Features to Model

In [7]:
sub_dataset = dataset[['replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'links', 'media', 'mentionedUsers', 'hashtags', 'Positive',
       'Negative', 'Neutral']]

In [10]:
#adding non-text features
X = np.concatenate((X, sub_dataset), 1)
X.shape

: 

: 

Run Model (Naive Bayes)

In [9]:
#training labels
y = dataset["Party"]

# Split into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25,
                                                    random_state = 0)

# Training and predicting
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

#Model performance
from sklearn.metrics import accuracy_score, classification_report
classification_report = classification_report(y_test, y_pred)

print('\n Accuracy: ', accuracy_score(y_test, y_pred))
print('\nClassification Report')
print('======================================================')
print('\n', classification_report)


 Accuracy:  0.8062797926406772

Classification Report

               precision    recall  f1-score   support

           0       0.81      0.87      0.84    113665
           1       0.80      0.71      0.76     82902

    accuracy                           0.81    196567
   macro avg       0.81      0.79      0.80    196567
weighted avg       0.81      0.81      0.80    196567

